# Import

In [1]:
pip install implicit

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/647.0 kB ? eta -:--:--
      --------------------------------------- 10.2/647.0 kB ? eta -:--:--
      --------------------------------------- 10.2/647.0 kB ? eta -:--:--
     -- ---------------------------------- 41.0/647.0 kB 279.3 kB/s eta 0:00:03
     -- ---------------------------------- 41.0/647.0 kB 279.3 kB/s eta 0:00:03
     ------ ----------------------------- 112.6/647.0 kB 467.6 kB/s eta 0:00:02
     --------- -------------------------- 174.1/647.0 kB 581.0 kB/s eta 0:00:01
     ------------------------------------ - 614.4/647.0 kB 1.9 MB/s eta 0:00:01
     ------------------------------------ - 614.4/647.0 kB 1.9 MB/s eta 0:00:01
     -------------------------------------- 647.0/647.0 kB 1.6 MB/s eta 0:00:00
     ---------------------------------------- 0.0/77.1 kB ? eta -:--:--
     ---------------------------------------- 77.1/77.1 kB ? eta 0:00

In [2]:
from utils.dataprep_utils import MovieDataset,neg_sampling
from utils.models_utils import ImplicitDLCRS,ExplicitDLCRS

from os import listdir
from os.path import isfile, join

from sklearn.metrics import mean_squared_error

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import pandas as pd
import numpy as np

from sklearn import model_selection, metrics, preprocessing
import matplotlib.pyplot as plt 
import implicit

C:\Users\CYTech Student\AppData\Roaming\Python\Python38\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data preparation

In [4]:
column_names = ['userId','movieId','rating','timestamp']
exp_df = pd.read_csv('./archive/ml-100k/u.data', sep='\t',header=None,names=column_names)
exp_df.head()

,userId,movieId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [6]:
imp_df = neg_sampling(exp_df,
                      percent_print=50)

imp_df.rename(columns={"interact":"rating"},inplace=True)
imp_df.head()

(943, 1682)


/content/drive/MyDrive/DL_CRS/utils/dataprep_utils.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nsamples['interact'] = nsamples.apply(lambda row: 1, axis=1)


processed ... 0.00% ...0.00secs
processed ... 49.95% ...0.07secs
processed ... 99.89% ...0.06secs


/content/drive/MyDrive/DL_CRS/utils/dataprep_utils.py:50: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  nsamples=nsamples.append(pd.DataFrame(nTempData, columns=["userId","movieId", "interact"]),ignore_index=True)


,userId,movieId,rating
0,195,241,1
1,185,301,1
2,21,376,1
3,243,50,1
4,165,345,1


In [7]:
# encode the user and movie id to start from 0 so we don't run into index out of bound with Embedding

lbl_user = preprocessing.LabelEncoder()
lbl_movie = preprocessing.LabelEncoder()

exp_df.userId = lbl_user.fit_transform(exp_df.userId.values)
exp_df.movieId = lbl_movie.fit_transform(exp_df.movieId.values)

imp_df.userId = lbl_user.fit_transform(imp_df.userId.values)
imp_df.movieId = lbl_movie.fit_transform(imp_df.movieId.values)

exp_train, exp_valid = model_selection.train_test_split(
    exp_df, test_size=0.2, random_state=42, stratify=exp_df.rating.values
)

imp_train, imp_valid = model_selection.train_test_split(
    imp_df, test_size=0.2, random_state=42, stratify=imp_df.rating.values
)

exp_train_dataset = MovieDataset(
    users=exp_train.userId.values,
    movies=exp_train.movieId.values,
    ratings=exp_train.rating.values
)

imp_train_dataset = MovieDataset(
    users=imp_train.userId.values,
    movies=imp_train.movieId.values,
    ratings=imp_train.rating.values
)

exp_valid_dataset = MovieDataset(
    users=exp_valid.userId.values,
    movies=exp_valid.movieId.values,
    ratings=exp_valid.rating.values
)

imp_valid_dataset = MovieDataset(
    users=imp_valid.userId.values,
    movies=imp_valid.movieId.values,
    ratings=imp_valid.rating.values
)

In [8]:
exp_train_loader = DataLoader(dataset=exp_train_dataset,
                              batch_size=64,
                              shuffle=True,
                              num_workers=4) 
 
exp_valid_loader = DataLoader(dataset=exp_valid_dataset,
                              batch_size=64,
                              shuffle=True,
                              num_workers=4)

imp_train_loader = DataLoader(dataset=imp_train_dataset,
                              batch_size=64,
                              shuffle=True,
                              num_workers=4) 
 
imp_valid_loader = DataLoader(dataset=imp_valid_dataset,
                              batch_size=64,
                              shuffle=True,
                              num_workers=4)

exp_dataiter = iter(exp_train_loader)
exp_dataloader_data = next(exp_dataiter) 
print(exp_dataloader_data)

imp_dataiter = iter(imp_train_loader)
imp_dataloader_data = next(imp_dataiter) 
print(imp_dataloader_data)

{'users': tensor([706, 220, 103, 585, 424, 872,  93,  94,  59, 758, 175, 880, 129, 532,
        596, 496,  58, 547, 621, 912, 504, 444, 386, 310, 726, 939, 451, 369,
        275, 278, 175, 471, 449, 921, 693, 393,  41, 893, 253,  73, 551, 379,
        832, 733, 416, 134, 462, 436, 256, 108, 893,  21, 108, 238, 657, 416,
        893, 850,  59, 888, 416, 344, 255,   4]), 'movies': tensor([ 653,  650,  983,   71,  683,  327, 1139,  513, 1019,  936,  507,   55,
          63,   24,  288,  385,  198,   49, 1206,  236,   94,  918,   19, 1478,
          68,  192,  606,  492,  247,  274,  880,   71,  382,  270, 1125,  558,
         195,  133,  185,  136,  287,  586,  695,  484,  401,  474, 1006,  134,
          58,  116, 1047,   16,  195,  835,  729, 1094,  332,  982,  750,  248,
         464,  314, 1423,  249]), 'ratings': tensor([4, 4, 1, 2, 2, 4, 2, 2, 4, 4, 3, 1, 5, 4, 5, 2, 4, 5, 2, 4, 4, 1, 4, 3,
        4, 3, 5, 5, 4, 3, 3, 5, 2, 3, 5, 4, 5, 4, 3, 3, 2, 4, 3, 5, 4, 4, 3, 4,
        5, 5,

# Modeling

## Call

In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [10]:
exp_model = ExplicitDLCRS(
    n_users=len(lbl_user.classes_),
    n_movies=len(lbl_movie.classes_),
    n_latents=5,
    n_layers=3,
    dropout=0.33,
).to(device)

imp_model = ImplicitDLCRS(
    n_users=len(lbl_user.classes_),
    n_movies=len(lbl_movie.classes_),
    n_latents=5,
    n_layers=3,
    dropout=0.33,
).to(device)

exp_optimizer = torch.optim.Adam(exp_model.parameters())  
imp_optimizer = torch.optim.Adam(exp_model.parameters())  

exp_sch = torch.optim.lr_scheduler.StepLR(exp_optimizer, step_size=3, gamma=0.7)
imp_sch = torch.optim.lr_scheduler.StepLR(imp_optimizer, step_size=3, gamma=0.7)

loss_ENTR = nn.CrossEntropyLoss()
loss_RMSE = nn.MSELoss()

In [11]:
print(exp_model)
print(imp_model)

ExplicitDLCRS(
  (user_embed): Embedding(943, 5)
  (movie_embed): Embedding(1682, 5)
  (fc_activation): ReLU()
  (fc_layers): ModuleList(
    (0): Linear(in_features=10, out_features=150, bias=True)
    (1): Linear(in_features=150, out_features=100, bias=True)
    (2): Linear(in_features=100, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=1, bias=True)
  )
  (do_layers): ModuleList(
    (0-2): 3 x Dropout(p=0.33, inplace=False)
  )
)
ImplicitDLCRS(
  (user_embed): Embedding(943, 5)
  (movie_embed): Embedding(1682, 5)
  (fc_activation): ReLU()
  (fc_layers): ModuleList(
    (0): Linear(in_features=10, out_features=150, bias=True)
    (1): Linear(in_features=150, out_features=100, bias=True)
    (2): Linear(in_features=100, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=2, bias=True)
  )
  (do_layers): ModuleList(
    (0-2): 3 x Dropout(p=0.33, inplace=False)
  )
)


## Training loop

In [12]:
exp_model_path = "./models/explicit/"

exp_list_saves = [f for f in listdir(exp_model_path) if isfile(join(exp_model_path, f))]

imp_model_path = "./models/implicit/"

imp_list_saves = [f for f in listdir(imp_model_path) if isfile(join(imp_model_path, f))]

In [13]:
if len(exp_list_saves) != 0 :
  exp_epochs_done = [int(save.split(".")[0]) for save in exp_list_saves]
  exp_most_recent_epoch = np.max(exp_epochs_done)
else :
  exp_most_recent_epoch = 0 

if len(imp_list_saves) != 0 :
  imp_epochs_done = [int(save.split(".")[0]) for save in imp_list_saves]
  imp_most_recent_epoch = np.max(imp_epochs_done)
else : 
  imp_most_recent_epoch = 0

### ExplicitDLCRS

In [14]:
epochs = 100
total_loss = []
plot_steps, print_steps = 5000, 5000
all_losses_list = [] 

exp_model.train() 
for epoch_i in range(exp_most_recent_epoch,epochs):
    saving_path = exp_model_path+str(epoch_i)+'.pt'
    if saving_path.split("/")[-1] in exp_list_saves :
      print(f"epoch {epoch_i} already done !")
      exp_model.load_state_dict(torch.load(saving_path))
    else :
      for i, train_data in enumerate(exp_train_loader):
          output = exp_model(train_data["users"].cuda(), 
                             train_data["movies"].cuda()) 
          
          rating = train_data["ratings"].view(output.shape[0], -1).to(torch.float32).cuda()
          loss = loss_RMSE(output, rating)
          total_loss.append(loss.sum().item())
          exp_optimizer.zero_grad()
          loss.backward()
          exp_optimizer.step()

      torch.save(exp_model.state_dict(),saving_path)
      avg_loss = np.mean(total_loss)
      print(f"epoch {epoch_i} loss is : {avg_loss}")
      print(f"model is saved for epoch {epoch_i}")
      all_losses_list.append(avg_loss)
      total_loss = [] 

epoch 99 already done !


In [15]:
torch.cuda.empty_cache()

### ImplicitDLCRS

In [16]:
epochs = 100
total_loss = []
plot_steps, print_steps = 5000, 5000
all_losses_list = [] 

imp_model.train() 
for epoch_i in range(imp_most_recent_epoch,epochs):
    saving_path = imp_model_path+str(epoch_i)+'.pt'
    if saving_path.split("/")[-1] in imp_list_saves :
      print(f"epoch {epoch_i} already done !")
      imp_model.load_state_dict(torch.load(saving_path))
    else :
      for i, train_data in enumerate(imp_train_loader):
          output = imp_model(train_data["users"].cuda(), 
                             train_data["movies"].cuda()) 
          
          rating = train_data["ratings"].view(output.shape[0], -1).to(torch.int64).cuda()
          loss = loss_ENTR(output, rating.reshape(output.shape[0]))
          total_loss.append(loss.sum().item())
          imp_optimizer.zero_grad()
          loss.backward()
          imp_optimizer.step()

      torch.save(imp_model.state_dict(),saving_path)
      avg_loss = np.mean(total_loss)
      print(f"epoch {epoch_i} loss is : {avg_loss}")
      print(f"model is saved for epoch {epoch_i}")
      all_losses_list.append(avg_loss)
      total_loss = [] 

epoch 0 loss is : 0.6936525440216065
model is saved for epoch 0
epoch 1 loss is : 0.6937459534883499
model is saved for epoch 1
epoch 2 loss is : 0.693608752655983
model is saved for epoch 2
epoch 3 loss is : 0.6937808111906052
model is saved for epoch 3
epoch 4 loss is : 0.6935935282230378
model is saved for epoch 4
epoch 5 loss is : 0.6936860323667526
model is saved for epoch 5
epoch 6 loss is : 0.6935326740980148
model is saved for epoch 6
epoch 7 loss is : 0.6936199694633484
model is saved for epoch 7
epoch 8 loss is : 0.693681895160675
model is saved for epoch 8
epoch 9 loss is : 0.6937492020845413
model is saved for epoch 9
epoch 10 loss is : 0.6936572890520096
model is saved for epoch 10
epoch 11 loss is : 0.6935763840436936
model is saved for epoch 11
epoch 12 loss is : 0.6937542610645294
model is saved for epoch 12
epoch 13 loss is : 0.6936070137739182
model is saved for epoch 13
epoch 14 loss is : 0.6936531247854233
model is saved for epoch 14
epoch 15 loss is : 0.69367742297

In [17]:
torch.cuda.empty_cache()

## Validation loop

### ExplicitDLCRS

In [23]:
exp_model_output_list = []
exp_target_rating_list = []

exp_model.eval()

with torch.no_grad():
    for i, batched_data in enumerate(exp_valid_loader): 
        model_output = exp_model(batched_data['users'].cuda(), 
                                 batched_data["movies"].cuda())
        
        exp_model_output_list.append(model_output.tolist())
        target_rating = batched_data["ratings"].tolist()
        exp_target_rating_list.append(target_rating)

def flatten(l):
    return [item for sublist in l for item in sublist]
    
exp_target_rating_list = flatten(exp_target_rating_list)
exp_model_output_list = flatten(exp_model_output_list)

# squared If True returns MSE value, if False returns RMSE value.
rms = mean_squared_error(exp_target_rating_list, 
                         exp_model_output_list, 
                         squared=False)
print(f"rms: {rms}")

rms: 0.9568647523927188


In [24]:
torch.cuda.empty_cache()

### ImplicitCLRS

In [25]:
from sklearn.metrics import mean_squared_error

imp_model_output_list = []
imp_target_rating_list = []

imp_model.eval()

with torch.no_grad():
    for i, batched_data in enumerate(imp_valid_loader): 
        model_output = imp_model(batched_data['users'].cuda(), 
                                 batched_data["movies"].cuda())
    
        imp_model_output_list.append(model_output.softmax(dim=1)[:,1].tolist())
        target_rating = batched_data["ratings"].tolist()
        imp_target_rating_list.append(target_rating)

def flatten(l):
    return [item for sublist in l for item in sublist]

imp_target_rating_list = flatten(imp_target_rating_list)
imp_model_output_list = flatten(imp_model_output_list)

# squared If True returns MSE value, if False returns RMSE value.
rms = mean_squared_error(imp_target_rating_list, 
                         imp_model_output_list,
                         squared=False)

print(f"rms: {rms}")

rms: 0.5001298608117489


In [26]:
torch.cuda.empty_cache()